<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [3]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range, todf
import tools.che as che
R=8.314 # J/(mol K)

In [4]:

p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    r.Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i # Approximation - water in the vapor phase is negligible
    W_n_vap = r.Pw * r.V_vap / (R * c.T)
    P = (c.air_n + W_n_vap) * R * c.T / r.V_vap
    return (s.P_f - P)**2

s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(360,300,600)


/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [6]:
# transform our model into one that takes a single array of adjustable variables
# for minimization.  Unnecessary for this simple model with just 1 variable, 
# but very helpful for more complex problems later.
vsc=VSC(v,s, model)
vsc.minimize(verbosity=0)

In [7]:
# Dataframe of adjustable variables
vsc.vdf

,vector1
,1
T,365.719235


In [8]:
# Dataframe of intermediate results
vsc.rdf

,vector1
,1
Pw,77275.424390
V_vap,0.008995


In [9]:
# Dataframe of static state parameters
vsc.sdf

,vector1
,1
W_tot,1.000000
V_tot,0.010000
P_i,100000.000000
P_f,200000.000000
T_i,298.000000
Vw_i,0.001005
Vair_i,0.008995
air_n,0.363057


In [10]:
def model2(c,r):
    Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i 
    r.W_n_vap = Pw * r.V_vap / (R * c.T) 
    P = (s.air_n + r.W_n_vap) * R * c.T / r.V_vap

    # Pressures and number of moles have vastly different order of magnitude
    # Scale each to order 1 before combining sum square deviations
    # Return r - store for intermediate results for reporting
    return ((s.P_f - P)/(s.P_f+P))**2 + ((r.W_n_vap - c.W_n_vap_desired)/(r.W_n_vap + 0.3))**2

In [11]:
s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.
s.W_n_vap_desired = 0.3

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, 300, 400)
v.V_tot = Range(0.015, 0., 0.03)

In [12]:
vsc=VSC(v,s, model2)
vsc.minimize()

0.10764694306583505
0.006849384312309694
0.006849384312309694
0.0006041999365723974
0.0006041999365723974
0.00011308656369463022
6.282090781480474e-08
6.282090781480474e-08
6.282090781480474e-08
7.758557021072123e-09
7.758557021072123e-09
3.4333762739728e-14
3.4333762739728e-14
3.4333762739728e-14
3.4333762739728e-14
3.4333762739728e-14
8.930295735303504e-20


In [13]:
vsc.vdf

,vector1
,1
T,370.005638
V_tot,0.011204


In [14]:
vsc.rdf

,vector1
,1
V_vap,0.010199
W_n_vap,0.300000


In [17]:
def model3(c, r):
    Pw = p.Pvap(c.T)[3]
    # volume of liquid water lost as water vapor ignored in model1
    r.v_w_vap = c.W_n_vap*18e-3/1e3
    V_vap = c.V_tot - (c.Vw_i - r.v_w_vap)
    r.W_n_vap = Pw * V_vap / (R * c.T)
    P = (s.air_n + r.W_n_vap) * R * c.T / V_vap

    # add consistency constraint for moles of water vapor
    return ((s.P_f - P)/(s.P_f + P))**2 + ((c.W_n_vap - r.W_n_vap)/(c.W_n_vap + r.W_n_vap))**2

In [18]:
s=DotMap()
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350., 300., 400.)
v.W_n_vap = Range(1., 0., 2.) # moles of water vapor

vsc=VSC(v,s, model3)
vsc.minimize(verbosity=0)

In [19]:
vsc.rdf

,vector1
,1
v_w_vap,0.000004
W_n_vap,0.228847


In [20]:
vsc.rdf.values # raw numbers
# only 4e-6 m3 of liquid water is lost to the vapor phase compared to the initial
# 1e-3 m3 of liquid water.

array([[4.11925524e-06],
       [2.28847490e-01]])

In [21]:
vsc.vdf

,vector1
,1
T,365.736688
W_n_vap,0.228848
